### Latent Dirichlet Allocation

In [145]:
import random
import pandas as pd
import numpy as np
from collections import Counter

- 토픽의 개수를 미리 결정(T=4) 하고
- 문서(documents) 에 대한 데이터 부여, 원래는 

In [146]:
T = 4

In [247]:
documents = [["은행", "플랫폼", "이자", "비이자", "Valuation", "성장성", "주가", "대출"], 
    ["자동차", "반도체", "성장성", "자율주행","당기순이익","칩"],
    ["CMO", "바이오", "코로나", "백신", "공정", "매출액", "성장성", "모더나", "바이러스"],
    ["실적", "이자", "비은행", "은행", "대출", "증권","비이자"],
    ["바이오시밀러", "코로나", "바이러스", "백신", "수익성", "바이오"],
    ["5G", "이동통신", "매출", "이익", "커머스", "지주"],
    ["스마트팩토리", "수익률", "통신", "IPTV"],
    ["GDP", "운용자산", "M&A", "코로나", "금리", "중앙은행"],
    ["IPTV", "턴어라운드", "5G", "이익", "인터넷", "통신"],
    ["반도체", "인공지능", "IP", "플랫폼", "칩", "자율주행"],
    ["원자현미경", "공정", "반도체", "성장", "수익성", "디스플레이"],
    ["로보택시", "모빌리티", "자율주행", "라이다", "자동차"],
    ["임상", "백신", "식약처", "3상", "결과발표", "모멘텀"]]

- $I(z_{n}^{(d)}=t)$ : document_topic_counts
- $\sum_{d=1}^{D}\sum_{n=1}^{N_{d}}I(w_{n}^{(d)}=m)I(z_{n}^{(d)}=t)$ : topic_word_counts

In [248]:
document_topic_counts = [Counter() for _ in documents]

topic_word_counts = [Counter() for _ in range(T)]

topic_counts = [0 for _ in range(T)]

- 각 문서(d)에 존재하는 단어의 개수 $N_{d}$ 집계

In [249]:
document_lengths = list(map(len, documents))

- 전체 문서의 개수 $(D)$ 집계

In [250]:
D = len(documents)

- 데이터에 존재하는 전체 단어의 Unique 개수 $(M)$ 집계

In [251]:
word_vec = list()
for d in range(D):
    N_d = list(map(len, documents))[d]
    for n in range(N_d):
        word_vec.append(documents[d][n])

unique_word_vec = set(word_vec)
M = len(unique_word_vec)

In [252]:
unique_word_vec = set(word_vec)
M = len(unique_word_vec)

#### Gibbs Sampling Step

- Gibbs Sampling 실행 과정에서 사용하게 될 conditional distribution 에 대한 코드 구현

$ p(z_{n}^{(d)}=t \vert z_{-n}^{(d)},\mathbf{w},\alpha,\beta) \propto \frac{ \beta_{m}+\sum_{d=1}^{D}\sum_{n=1}^{N_{d}}I(w_{n}^{(d)}=m)I(z_{n}^{(d)}=t)}{ \sum_{m=1}^{M}\left(\beta_{m}+\sum_{d=1}^{D}\sum_{n=1}^{N_{d}}I(w_{n}^{(d)}=m)I(z_{n}^{(d)}=t) \right) } \times \left( \alpha_{t}+ \sum_{n=1}^{N_{d}} I(z_{n}^{(d)}=t) \right) $

In [253]:
def topic_weight(d, word, topic):
    
    alpha = 1/T
    beta  = 1/M
    
    return (document_topic_counts[d][topic]+alpha)*((topic_word_counts[topic][word]+beta)/(topic_counts[topic]+M*beta))

In [254]:
def choose_new_topic(d, word) :
    weights = np.zeros(T)
    for topic in range(T) :
        weights[topic] = topic_weight(d, word, topic) 
    
    total = weights.sum()
    t = np.argmax(np.random.multinomial(1, pvals = weights/total))
    return t

- 문서에 존재하는 각 단어들에 대해 토픽 할당
- Gibbs Sampling 수행을 위해 각 단어에 대해 토픽 최초값을 임의로 할당

In [255]:
document_topics = [[random.randrange(T) for word in document] for document in documents]
document_topics

[[2, 0, 3, 1, 3, 1, 1, 2],
 [1, 1, 2, 0, 1, 2],
 [3, 0, 0, 3, 3, 0, 1, 1, 0],
 [1, 0, 2, 0, 0, 0, 2],
 [2, 1, 3, 0, 1, 1],
 [2, 1, 0, 1, 2, 2],
 [3, 2, 0, 2],
 [0, 1, 2, 1, 1, 0],
 [3, 1, 3, 0, 3, 2],
 [1, 1, 1, 1, 0, 3],
 [1, 2, 0, 1, 0, 2],
 [1, 3, 2, 2, 3],
 [2, 2, 0, 0, 2, 3]]

- zip은 두 개의 list의 값을 병렬 추출
- 두 개의 list가 있을 때 각 인덱스에 있는 값들을 뽑아주는 것이 zip
- 문서에 대해 for문을 돌리면서 개수를 Count

In [256]:
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] = document_topic_counts[d][topic] + 1 
        topic_word_counts[topic][word] = topic_word_counts[topic][word] + 1
        topic_counts[topic] = topic_counts[topic] + 1

In [257]:
document_topic_counts

[Counter({2: 2, 0: 1, 3: 2, 1: 3}),
 Counter({1: 3, 2: 2, 0: 1}),
 Counter({3: 3, 0: 4, 1: 2}),
 Counter({1: 1, 0: 4, 2: 2}),
 Counter({2: 1, 1: 3, 3: 1, 0: 1}),
 Counter({2: 3, 1: 2, 0: 1}),
 Counter({3: 1, 2: 2, 0: 1}),
 Counter({0: 2, 1: 3, 2: 1}),
 Counter({3: 3, 1: 1, 0: 1, 2: 1}),
 Counter({1: 4, 0: 1, 3: 1}),
 Counter({1: 2, 2: 2, 0: 2}),
 Counter({1: 1, 3: 2, 2: 2}),
 Counter({2: 3, 0: 2, 3: 1})]

In [258]:
for epoch in range(100): # repetition
    for d in range(D): # each documnet
        for i, (word, topic) in enumerate(zip(documents[d],document_topics[d])):
            
            # gibbs sampling: 특정 하나의 topic assignment z를 제거하고 나머지들(-z)의 조건부 확률  
            
            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1 # 문서별 토픽 갯수
            topic_word_counts[topic][word] -= 1 # 토픽별 단어 갯수
            topic_counts[topic] -= 1 # 토픽별 카운트
            document_lengths[d] -= 1 # 문서별 단어갯수
            
            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            
            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1 # 문서별 토픽 갯수
            topic_word_counts[new_topic][word] += 1 # 토픽별 단어 갯수
            topic_counts[new_topic] += 1 # 토픽별 카운트
            document_lengths[d] += 1 # 문서별 단어갯수

In [259]:
df = pd.DataFrame(columns=['Topic1','Topic2','Topic3','Topic4'], index=['Top'+str(i) for i in range(1,6)])

for k, word_counts in enumerate(topic_word_counts):
    for ix, (word, count) in enumerate(word_counts.most_common(6)): # 각 토픽별로 top 10 단어
            df.loc['Top'+str(ix+1),'Topic'+str(k+1)] = word+'({})'.format(count)

In [260]:
df

,Topic1,Topic2,Topic3,Topic4
Top1,이자(2),성장성(3),백신(3),IPTV(2)
Top2,은행(2),반도체(3),코로나(3),5G(2)
Top3,대출(2),자율주행(3),바이오(2),이익(2)
Top4,비이자(2),자동차(2),수익성(2),통신(2)
Top5,증권(1),플랫폼(2),바이러스(2),스마트팩토리(1)
Top6,Valuation(1),공정(2),바이오시밀러(1),인터넷(1)
